<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EA%B3%BC%EC%A0%95/%EC%9B%90%EB%B3%B8C%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC%EC%9D%98_%EC%82%AC%EB%B3%B8%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 47.1 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=e6f09fd66dcb331b61e856cd4c413264a40989035d54ec61665a898abbafe3cf
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.1 MB/s 
     |████████████████████████████████| 769 kB 5.0 MB/s 
     |████████████████████████████████| 3.0 MB 20.2 MB/s 
     |████████████████████████████████| 895 kB 62.7 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-zw0u790m
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/p

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화		'# df_G_inc.csv'    G_inc_model.pt   Q_inc_model.pt
'Colab Notebooks'	 df_G_inc.csv	    I_ori_model.pt   S_ori_model.pt
'답안 작성용 파일.csv'	 df_J_inc.csv	    J_inc_model.pt   데이터.zip
 df_A_inc.csv		 df_K_inc.csv	   'My Drive'
 df_B_inc.csv		 df_N_inc.csv	    N_inc_model.pt
 df_E_inc.csv		 F_inc_model_2.pt   P_inc_model.pt


In [4]:
!unzip /content/gdrive/'My Drive'/산업분류자동화/'모델 개발용 자료'.zip -d /content/dataset
!unzip /content/gdrive/'My Drive'/산업분류자동화/'실습용 자료'.zip -d /content/dataset

Archive:  /content/gdrive/My Drive/산업분류자동화/모델 개발용 자료.zip
  inflating: /content/dataset/2. ╕Ё╡и┐ы░│╣▀┐ы└┌╖с.csv  
Archive:  /content/gdrive/My Drive/산업분류자동화/실습용 자료.zip
1. 실습용자료.txt:  mismatching "local" filename (1. ьЛдьК╡ьЪйьЮРыгМ.txt),
         continuing with "central" filename version
  inflating: /content/dataset/1. 실습용자료.txt  


In [5]:
import pandas as pd
df=pd.read_csv('/content/dataset/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/content/gdrive/MyDrive/답안 작성용 파일.csv', encoding = "euc-kr")

In [6]:
df=df[df['digit_1']=='C']
ts=ts[ts['digit_1']=='C']

In [7]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [8]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000006,C,29,291,철,절삭.용접,카프라배관자재
1,id_0000008,C,10,107,쌀을 가지고,가공하여,떡제조
2,id_0000011,C,10,102,멸치,"입고, 가공",NaN
3,id_0000021,C,10,107,쌀을가지고,가루로 분쇄하여 떡을 쪄서,백설기 꿀떡 절편판매
4,id_0000027,C,22,221,고무,절단가공,"가스켓,다이아후레임"
5,id_0000030,C,29,292,Resim,"재료구입, 세미앗세이",NaN
6,id_0000044,C,25,259,금속재료,주문에의한 절삭가공,공작기계
7,id_0000053,C,22,222,공장에서,다른사업체의 요청에 의해,부품제조
8,id_0000056,C,32,320,쇼파제조사업장에서,주문에 의해,쇼파 제조
9,id_0000081,C,14,141,원단을 가지고,재단 봉제하여,아동복


In [9]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000006,C,29,291,철 절삭.용접 카프라배관자재
1,id_0000008,C,10,107,쌀을 가지고 가공하여 떡제조
2,id_0000011,C,10,102,"멸치 입고, 가공"
3,id_0000021,C,10,107,쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매
4,id_0000027,C,22,221,"고무 절단가공 가스켓,다이아후레임"
...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스,다이오드,칩 조립 전원공급장치(파워써플라이)"
105188,id_0999976,C,25,259,기타임가공
105189,id_0999986,C,10,106,벼를 가지고 벼를 도정하여 소비자에게 판매
105190,id_0999994,C,32,320,"카본열선,퀼팅구조체 조립.배열.봉제하여 의료용매트리스"


In [10]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [11]:
df.shape

(105192, 5)

In [12]:
df['digit_3'].value_counts()

259    12876
107     9214
292     7163
222     4935
291     4924
       ...  
304       22
301       20
191       20
266        8
120        3
Name: digit_3, Length: 85, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [14]:
df.loc[(df['digit_3']==101), 'digit_3']=0
df.loc[(df['digit_3']==102), 'digit_3']=1
df.loc[(df['digit_3']==103), 'digit_3']=2
df.loc[(df['digit_3']==104), 'digit_3']=3
df.loc[(df['digit_3']==105), 'digit_3']=4
df.loc[(df['digit_3']==106), 'digit_3']=5
df.loc[(df['digit_3']==107), 'digit_3']=6
df.loc[(df['digit_3']==108), 'digit_3']=7
df.loc[(df['digit_3']==111), 'digit_3']=8
df.loc[(df['digit_3']==112), 'digit_3']=9
df.loc[(df['digit_3']==120), 'digit_3']=10
df.loc[(df['digit_3']==131), 'digit_3']=11
df.loc[(df['digit_3']==132), 'digit_3']=12
df.loc[(df['digit_3']==133), 'digit_3']=13
df.loc[(df['digit_3']==134), 'digit_3']=14
df.loc[(df['digit_3']==139), 'digit_3']=15
df.loc[(df['digit_3']==141), 'digit_3']=16
df.loc[(df['digit_3']==142), 'digit_3']=17
df.loc[(df['digit_3']==143), 'digit_3']=18
df.loc[(df['digit_3']==144), 'digit_3']=19
df.loc[(df['digit_3']==151), 'digit_3']=20
df.loc[(df['digit_3']==152), 'digit_3']=21
df.loc[(df['digit_3']==161), 'digit_3']=22
df.loc[(df['digit_3']==162), 'digit_3']=23
df.loc[(df['digit_3']==163), 'digit_3']=24
df.loc[(df['digit_3']==171), 'digit_3']=25
df.loc[(df['digit_3']==172), 'digit_3']=26
df.loc[(df['digit_3']==179), 'digit_3']=27
df.loc[(df['digit_3']==181), 'digit_3']=28
df.loc[(df['digit_3']==182), 'digit_3']=29
df.loc[(df['digit_3']==191), 'digit_3']=30
df.loc[(df['digit_3']==192), 'digit_3']=31
df.loc[(df['digit_3']==201), 'digit_3']=32
df.loc[(df['digit_3']==202), 'digit_3']=33
df.loc[(df['digit_3']==203), 'digit_3']=34
df.loc[(df['digit_3']==204), 'digit_3']=35
df.loc[(df['digit_3']==205), 'digit_3']=36
df.loc[(df['digit_3']==211), 'digit_3']=37
df.loc[(df['digit_3']==212), 'digit_3']=38
df.loc[(df['digit_3']==213), 'digit_3']=39
df.loc[(df['digit_3']==221), 'digit_3']=40
df.loc[(df['digit_3']==222), 'digit_3']=41
df.loc[(df['digit_3']==231), 'digit_3']=42
df.loc[(df['digit_3']==232), 'digit_3']=43
df.loc[(df['digit_3']==233), 'digit_3']=44
df.loc[(df['digit_3']==239), 'digit_3']=45
df.loc[(df['digit_3']==241), 'digit_3']=46
df.loc[(df['digit_3']==242), 'digit_3']=47
df.loc[(df['digit_3']==243), 'digit_3']=48
df.loc[(df['digit_3']==251), 'digit_3']=49
df.loc[(df['digit_3']==252), 'digit_3']=50
df.loc[(df['digit_3']==259), 'digit_3']=51
df.loc[(df['digit_3']==261), 'digit_3']=52
df.loc[(df['digit_3']==262), 'digit_3']=53
df.loc[(df['digit_3']==263), 'digit_3']=54
df.loc[(df['digit_3']==264), 'digit_3']=55
df.loc[(df['digit_3']==265), 'digit_3']=56
df.loc[(df['digit_3']==266), 'digit_3']=57
df.loc[(df['digit_3']==271), 'digit_3']=58
df.loc[(df['digit_3']==272), 'digit_3']=59
df.loc[(df['digit_3']==273), 'digit_3']=60
df.loc[(df['digit_3']==274), 'digit_3']=61
df.loc[(df['digit_3']==281), 'digit_3']=62
df.loc[(df['digit_3']==282), 'digit_3']=63
df.loc[(df['digit_3']==283), 'digit_3']=64
df.loc[(df['digit_3']==284), 'digit_3']=65
df.loc[(df['digit_3']==285), 'digit_3']=66
df.loc[(df['digit_3']==289), 'digit_3']=67
df.loc[(df['digit_3']==291), 'digit_3']=68
df.loc[(df['digit_3']==292), 'digit_3']=69
df.loc[(df['digit_3']==301), 'digit_3']=70
df.loc[(df['digit_3']==302), 'digit_3']=71
df.loc[(df['digit_3']==303), 'digit_3']=72
df.loc[(df['digit_3']==304), 'digit_3']=73
df.loc[(df['digit_3']==311), 'digit_3']=74
df.loc[(df['digit_3']==312), 'digit_3']=75
df.loc[(df['digit_3']==313), 'digit_3']=76
df.loc[(df['digit_3']==319), 'digit_3']=77
df.loc[(df['digit_3']==320), 'digit_3']=78
df.loc[(df['digit_3']==331), 'digit_3']=79
df.loc[(df['digit_3']==332), 'digit_3']=80
df.loc[(df['digit_3']==333), 'digit_3']=81
df.loc[(df['digit_3']==334), 'digit_3']=82
df.loc[(df['digit_3']==339), 'digit_3']=83
df.loc[(df['digit_3']==340), 'digit_3']=84

In [15]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['철 절삭.용접 카프라배관자재', '68'], ['쌀을 가지고 가공하여 떡제조', '6'], ['멸치 입고, 가공 ', '1'], ['쌀을가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편판매', '6'], ['고무 절단가공 가스켓,다이아후레임', '40']]


## Train data & test data

In [16]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [19]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [20]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [21]:
data_train[0]

(array([   2, 4473, 7688, 4067, 6515,  517,   54,  517, 5330, 5452, 2641,
        7003, 7980, 7234, 7253,    3,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(16, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 51)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=85,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [25]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1480 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 4.485278606414795 train acc 0.046875
epoch 1 batch id 201 loss 3.7978267669677734 train acc 0.07828047263681592
epoch 1 batch id 401 loss 2.6966207027435303 train acc 0.1719529301745636
epoch 1 batch id 601 loss 2.0286998748779297 train acc 0.26546900998336104
epoch 1 batch id 801 loss 1.8884390592575073 train acc 0.3394975031210986
epoch 1 batch id 1001 loss 1.5572668313980103 train acc 0.3979301948051948
epoch 1 batch id 1201 loss 1.5781230926513672 train acc 0.4450718151540383
epoch 1 batch id 1401 loss 1.1963268518447876 train acc 0.4796685403283369
epoch 1 train acc 0.4918074324324324


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.785290404040404


  0%|          | 0/1480 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.8009027242660522 train acc 0.8125
epoch 5 batch id 201 loss 0.44862186908721924 train acc 0.8564987562189055
epoch 5 batch id 401 loss 0.5086261034011841 train acc 0.8587515586034913
epoch 5 batch id 601 loss 0.3686373233795166 train acc 0.8592190099833611
epoch 5 batch id 801 loss 0.5044829845428467 train acc 0.8595310549313359
epoch 5 batch id 1001 loss 0.5324227213859558 train acc 0.8604832667332667
epoch 5 batch id 1201 loss 0.664458692073822 train acc 0.8620680682764363
epoch 5 batch id 1401 loss 0.4214050769805908 train acc 0.8622301034975017
epoch 5 train acc 0.8626055743243243


  0%|          | 0/165 [00:00<?, ?it/s]

epoch 5 test acc 0.7874368686868688


In [26]:
#학습 모델 저장
torch.save(model, 'C_ori_model.pt')